### Pulling in LEV files and to Construct Deployment, Well, and Data Models

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os, glob, linecache, uuid
import xml.etree.ElementTree as ETE
import seaborn as sns
# import matplotlib.pyplot as plt
import shutil
from scipy.stats import zscore
import re
import ipdb

uuid_gen = uuid.uuid4()

In [2]:
data_dir = "C:/Users/jinsu.elhance/Box/Wells/WellsDatasheets"

In [3]:
LEV_dir_list = glob.glob(f"{data_dir}/*/*.lev")
XLE_dir_list = glob.glob(f"{data_dir}/*/*.xle") + glob.glob(f"{data_dir}/*/*/*.xle")
XLSX_dir_list = glob.glob(f"{data_dir}/WellsData_Dudek_2009-2011/*.xlsx")
CSV_dir_2017_2018_list = glob.glob(f"{data_dir}/WellsData_2017-2018/*.csv")

In [17]:
processed = set([os.path.basename(file) for file in LEV_dir_list + XLE_dir_list + XLSX_dir_list + CSV_dir_2017_2018_list])
unprocessed_list = [glob.glob(f"{data_dir}/*/*.{ext}") for ext in ["xlsx", "lev", "xle", "csv"]] + [glob.glob(f"{data_dir}/*/*/*.xle")]
unprocessed = set()
for ext in unprocessed_list:
    unprocessed.update([os.path.basename(file) for file in ext])

unprocessed = unprocessed.difference(processed)

with open(f"{data_dir}/SynthesisStatus.txt", "w") as output:
    output.write("Unprocessed datasheets \n")
    for file in unprocessed:
        output.write(str(file) + "\n")
    output.write("Emails\n")
    output.write("\nProcessed datasheets \n")
    for file in processed:
        output.write(str(file) + "\n")

In [4]:
def find_data_rows(lines):
    i = 0 
    while i < len(lines):
        if lines[i] == "[Data]\r\n":
            return i 
        i += 1
        
def getXLEMetadata(xle_dict) :
    _metadata = {}
    _instrumentInfo = xle_dict.get("Instrument_info")
    _instrumentDataInfo = xle_dict.get("Instrument_info_data_header")
    
    _metadata['Instrumenttype'] = _instrumentInfo.get('Instrument_type')
    _metadata['Instrumentstate'] = _instrumentInfo.get('Instrument_state')
    _metadata['Serialnumber'] = _instrumentInfo.get("Serial_number")
    _metadata['Location'] = (_instrumentDataInfo.get('Location') or "Unknown").replace("/", "").replace("#", "").replace(" ","").lower()
    _metadata['SampleRate'] = _instrumentDataInfo.get("Sample_rate")
    _metadata['SampleMode'] = _instrumentDataInfo.get("Sample_mode")
    _metadata['Altitude'] = xle_dict.get('Ch1_data_header', {}).get('Parameters', {}).get('Altitude', 0)
    _metadata['StartTime'] = _instrumentDataInfo.get('Start_time') or 'Unknown'
    _metadata['StopTime'] = _instrumentDataInfo.get('Stop_time') or 'Unknown'
    _metadata['LevelUnit'] = xle_dict.get('Ch1_data_header', {}).get('Unit', 'unit')
    _metadata['TemperatureUnit'] = xle_dict.get('Ch2_data_header', {}).get('Unit', 'unit')[-1]
    
    return _metadata

In [5]:
WellDevices = pd.DataFrame()
WellData = {}

In [28]:
for lev in LEV_dir_list:
    with open(lev, newline="\n") as lev_text:
        lev_lines = lev_text.readlines()
        _dataStart = find_data_rows(lev_lines) + 2
        
        #Pull out metadata from datafile
        _metadata = {}
        for _mdx in lev_lines[10:_dataStart]:
            _spltKeyDat = _mdx.replace(" ", "").strip().split("=")
            if len(_spltKeyDat) > 1:
                if _spltKeyDat[0] == "Unit" and "LevelUnit" in _metadata.keys():
                    _spltKeyDat[0] = "TemperatureUnit"
                elif _spltKeyDat[0] == "Unit":
                    _spltKeyDat[0] = "LevelUnit"
                _metadata[_spltKeyDat[0]] = _spltKeyDat[1]

        #Which well?
        location = (_metadata.get("Location") or "Unknown").replace("/", "").replace("#", "").replace(" ","").lower()
        instrumentType = _metadata.get('Instrumenttype' or "Unknown")
        
        #Create folder for storing data
        if not os.path.exists(f"{data_dir}/HistoricalWellSynthesis/Data/{location}"):
            os.makedirs(f"{data_dir}/HistoricalWellSynthesis/Data/{location}")
            
        #Copy datasheet to data store directory
        if not os.path.exists(f"{data_dir}/HistoricalWellSynthesis/Data/{location}/{os.path.basename(lev)}"):
            shutil.copy2(lev, f"{data_dir}/HistoricalWellSynthesis/Data/{location}/{os.path.basename(lev)}")
            
        ## Find Data Pointer
        _df = pd.read_fwf(lev, skiprows=_dataStart, names=["date", "time", "level", "temperature"], encoding='iso-8859-1')
        _df = _df.iloc[:-1]
        _df = _df.set_index(_df.agg(('{0[date]}{0[time]}' + f"{location}{instrumentType}").format, axis=1).apply(lambda x: hash(x)))
        _df['date'] = pd.to_datetime(_df['date'])
        
        #Any data after 2020 will be barometrically corrected.
        if "2020" in lev or "2021" in lev:
            _df['baro_corrected'] = False
        else:
            _df['baro_corrected'] = True
        
        #Update Metadata
        _metadata['dataStartDate'] = min(_df['date'])
        _metadata['dataEndDate'] = max(_df['date'])
        _metadata['TemperatureUnit'] = _metadata['TemperatureUnit'][-1]
        _metadata["Location"] = location
        
        #Apply conversions
        if _metadata.get("TemperatureUnit") == "C":
            _df['temperature_c'] = _df['temperature'].astype(float)
            _df['temperature_f'] = _df['temperature'].astype(float) * 1.8 + 32
        else:
            _df['temperature_f'] = _df['temperature']

        if _metadata.get("LevelUnit", "unit") == "m":
            _df['level'] = _df['level'].astype(float) / 3.28084
                    
        #Save data to dataframes
        if location in WellData.keys():
            WellData[location] = pd.concat([WellData[location], _df], axis=0)
        else:
            WellData[location] = _df

        WellDevices = pd.concat([WellDevices, pd.DataFrame(_metadata, index=[instrumentType+location])], ignore_index = True)

KeyboardInterrupt: 

In [25]:
import xml_to_dict

parser = xml_to_dict.XMLtoDict()

#Similar data-read to the LEV read. However, makes use of a metadata parsing helper.
for xle in XLE_dir_list:
    
    with open(xle, newline="\n") as xle_text:
        xle_content = xle_text.read()
        xle_dict = parser.parse(xle_content)['Body_xle']
        
        _metadata = getXLEMetadata(xle_dict)
        location = _metadata['Location']
        
        if not os.path.exists(f"{data_dir}/HistoricalWellSynthesis/Data/{location}"):
            os.makedirs(f"{data_dir}/HistoricalWellSynthesis/Data/{location}")
            
        if not os.path.exists(f"{data_dir}/HistoricalWellSynthesis/Data/{location}/{os.path.basename(xle)}"):
            shutil.copy2(xle, f"{data_dir}/HistoricalWellSynthesis/Data/{location}/{os.path.basename(xle)}")
            
        _df = pd.DataFrame(xle_dict.get('Data', {}).get('Log', {}))
        _df = _df.set_index(_df.agg(('{0[Date]}{0[Time]}' + f"{location}{instrumentType}").format, axis=1).apply(lambda x: hash(x)))
        _df = _df.rename(columns = {"Date": 'date', "Time":'time', "ch1":"level", 'ch2':'temperature' }).drop(['ms', '@id'], axis=1)
        
        if _metadata['TemperatureUnit'] == "C":
            _df['temperature_c'] = _df['temperature'].astype(float)
            _df['temperature_f'] = _df['temperature'].astype(float) * 1.8 + 32
        else:
            _df['temperature_f'] = _df['temperature']

        if _metadata['LevelUnit'] =="m":
            _df['level'] = _df['level'].astype(float) / 3.28084
            
        _metadata['dataStartDate'] = min(_df['date'])
        _metadata['dataEndDate'] = max(_df['date'])
        
        #Any data after 2020 will need to be barometrically corrected.
        if "2020" in xle or "2021" in xle:
            _df['baro_corrected'] = False
        else:
            _df['baro_corrected'] = True
        
        if location in WellData.keys():
            WellData[location] = pd.concat([WellData[location], _df], axis=0)
        else: 
            WellData[location] = _df
            
        WellDevices = pd.concat([WellDevices, pd.DataFrame(_metadata, index=[instrumentType+location])], ignore_index = True)

In [29]:
for xlsx in XLSX_dir_list:
    location = xlsx.split("\\")[1][:-5].replace("_","").replace(" ","").lower()

    if not os.path.exists(f"{data_dir}/HistoricalWellSynthesis/Data/{location}"):
        os.makedirs(f"{data_dir}/HistoricalWellSynthesis/Data/{location}")
    
    if not os.path.exists(f"{data_dir}/HistoricalWellSynthesis/Data/{location}/{os.path.basename(xlsx)}"):
        shutil.copy2(xlsx, f"{data_dir}/HistoricalWellSynthesis/Data/{location}/{os.path.basename(xlsx)}")
        
    _df = pd.read_excel(xlsx, sheet_name = "data")
    _df = _df[["DATE TIME", "Level (ft)", "Temperature °C", "DATE TIME.1", "Barologger Level (ft)", "Barologger Corrected for Elevation", "Temperature °C.1", "Corrected Water Level (ft H2O)", "Groundwater Elevation (ft MSL)"]]

    _df.loc[_df['DATE TIME.1'].isna(), 'DATE TIME.1'] = _df.loc[_df['DATE TIME.1'].isna(), 'DATE TIME']
    _df.loc[_df['DATE TIME'].isna(), 'DATE TIME'] = _df.loc[_df['DATE TIME'].isna(), 'DATE TIME.1']

    _df = _df.loc[_df['Temperature °C'].apply(type) == float] 

    _df['date'] = _df['DATE TIME'].dt.date
    _df['time'] = _df['DATE TIME'].dt.time
    _df['temperature_f'] = _df['Temperature °C'].astype(float)  * 1.8 + 32
    _df['temperature_barometer_f'] = _df['Temperature °C.1'].astype(float) * 1.8 + 32
    _df['baro_corrected'] = True
    
    _df = _df.rename(columns = {
        'Level (ft)': 'level', 
        'temperature_f':'temperature_f', 
        'Temperature °C': 'temperature_c',
        'Barologger Level (ft)':'barologger_level', 
        'Barologger Corrected for Elevation': 'barologger_level_c_elevation',
        'Temperature °C.1': 'temperature_barometer_c',
        'Corrected Water Level (ft H2O)': 'level_corrected',
        'Groundwater Elevation (ft MSL)': 'groundwater_elevation'}
        )[['date','time','level','temperature_f','barologger_level','barologger_level_c_elevation','temperature_barometer_f','level_corrected','groundwater_elevation']]

    #Some barologgers collect at longer intervals, forward fill values.
    _df['barologger_level'] = _df['barologger_level'].ffill(axis=0)
    _df['barologger_level_c_elevation'] = _df['barologger_level_c_elevation'].ffill(axis=0) #barologger level, corrected for elevation.
    _df['temperature_barometer_f']  = _df['temperature_barometer_f'].ffill(axis=0)
    _df['level_corrected'] = _df['level'] - _df['barologger_level_c_elevation']
    
    transducer_depth_f = _df['groundwater_elevation'].values[0] - _df['level_corrected'].values[0]
    _df['groundwater_elevation'] = _df['level_corrected'] + transducer_depth_f
    
    if location in WellData.keys():
        WellData[location] = pd.concat([WellData[location], _df], axis=0)
    else:
        WellData[location] = _df
        
#Not updating devices from xlsxs

In [8]:
# WellDevices = pd.read_csv(f"{data_dir}/HistoricalWellSynthesis/Data/WellDevices.csv")
# full_well_data = pd.read_csv(f"{data_dir}/HistoricalWellSynthesis/Data/full_well_data.csv")

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgeopy\lib\site-packages\IPython\core\interactiveshell.py:3460: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:
#Find well elevations from Devices 2020 csv.
Devices_2020 = pd.read_csv("C:/Users/jinsu.elhance/Box/Wells/JLDP_Wells_Sensor_List_2020_08_19.csv")
Devices_2020['Well_Name_std'] = Devices_2020['Well_Name'].str.replace("/", "").str.replace("#", "").str.replace(" ","").str.replace(".", "").str.lower()
Devices_2020['Well_Name_std']
well_elevations_ft = (Devices_2020.groupby("Well_Name_std")['Elevation_m'].first() * 3.28084).to_dict()

#Add elevations for missing wells.
well_elevations_ft['tinta3bl'] = well_elevations_ft['tinta3']
well_elevations_ft['lowerjalamavaqueros'] = 492.16
well_elevations_ft['quailcanyon1'] = well_elevations_ft['quailcanyon']
well_elevations_ft['tinta11b'] = well_elevations_ft['tinta11a']
well_elevations_ft['tinta5b'] = well_elevations_ft['tinta5']
well_elevations_ft['oaks3bbl'] = well_elevations_ft['oaks3']
well_elevations_ft['venidido2'] = well_elevations_ft['venadito2'] #Is venidido a different well?
well_elevations_ft['escondido3a'] = well_elevations_ft['escondido3']
well_elevations_ft['unknown'] = 0
well_elevations_ft['oaks3a'] = well_elevations_ft['oaks3']

#The elevation of K7AZ Lompoc MesoWest Weather Station
baro_elevation_ft = 1529

<ipython-input-30-c594f08cb2d9>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Devices_2020['Well_Name_std'] = Devices_2020['Well_Name'].str.replace("/", "").str.replace("#", "").str.replace(" ","").str.replace(".", "").str.lower()


In [31]:
WellDevices = WellDevices.set_index(WellDevices["Instrumenttype"] + WellDevices["Location"])
WellDevices['Altitude'] = WellDevices['Altitude'].astype(str).apply(lambda x: x.replace("ft", ""))
WellDevices = WellDevices.drop_duplicates()
WellDevices.groupby(["Location", "dataStartDate"]).first()

WellDevices = WellDevices[[
    "Instrumenttype",
    "Serialnumber",
    "Location",
    "SampleRate",
    "Altitude",
    "StartTime",
    "StopTime",
    "LevelUnit",
    "TemperatureUnit",
    "dataStartDate",
    "dataEndDate",
]]

WellDevices.to_csv(f"{data_dir}/HistoricalWellSynthesis/Data/WellDevices.csv")

In [32]:
#construct full well data
full_well_data = pd.DataFrame()
for Well, Data in WellData.items():
    Data["well"] = Well
    full_well_data = pd.concat([full_well_data, Data])

In [33]:
#Ensure all entries have an associated timestamp, otherwise remove.
full_well_data = full_well_data.loc[~full_well_data['time'].isna()]

#Convert datetime column to a datetime data type
full_well_data['datetime'] = pd.to_datetime(full_well_data['date'].astype(str) + " " + full_well_data['time'].astype(str));

#Change column types to numeric
full_well_data['level'] = full_well_data['level'].astype(float)
full_well_data['temperature_f'] = full_well_data['temperature_f'].astype(float)
full_well_data['temperature_c']= full_well_data['temperature_c'].astype(float)

#For any missing level values, take corrected level. Some CSVs only contained "corrected levels"
full_well_data.loc[full_well_data['level'].isna(), 'level'] = full_well_data.loc[full_well_data['level'].isna()]['level_corrected']
full_well_data = full_well_data.drop_duplicates()
full_well_data['datetime_hr_10min'] = full_well_data.datetime.astype(str).apply(lambda x: x[:-4])

# full_well_data.to_csv(f"{data_dir}/HistoricalWellSynthesis/Data/full_well_data.csv")